In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['abcnews-date-text.csv']


In [2]:
import pandas as pd

data = pd.read_csv('../input/abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [3]:
len(documents)

1103663

In [4]:
documents[:5]

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


In [5]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(0)

/opt/conda/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [6]:
import nltk

In [7]:
print(WordNetLemmatizer().lemmatize('went', pos='v'))

go


In [8]:
stemmer = SnowballStemmer('english')
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data = {'original word': original_words, 'stemmed': singles})

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [9]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [10]:
doc_sample = documents[documents['index'] == 4310].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['rain', 'helps', 'dampen', 'bushfires']


 tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


In [11]:
processed_docs = documents['headline_text'].map(preprocess)

In [12]:
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [13]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [14]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [15]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [16]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(76, 1), (112, 1), (483, 1), (4014, 1)]

In [17]:
bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 76 ("bushfir") appears 1 time.
Word 112 ("help") appears 1 time.
Word 483 ("rain") appears 1 time.
Word 4014 ("dampen") appears 1 time.


In [18]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [19]:
corpus_tfidf = tfidf[bow_corpus]

In [20]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5892908867507543),
 (1, 0.38929654337861147),
 (2, 0.4964985175717023),
 (3, 0.5046520327464028)]


In [21]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [22]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.035*"say" + 0.031*"govern" + 0.020*"chang" + 0.016*"rural" + 0.013*"minist" + 0.012*"farmer" + 0.011*"support" + 0.011*"need" + 0.011*"labor" + 0.010*"feder"
Topic: 1 
Words: 0.034*"trump" + 0.026*"melbourn" + 0.019*"world" + 0.018*"countri" + 0.011*"abus" + 0.010*"game" + 0.010*"victorian" + 0.009*"liber" + 0.009*"race" + 0.008*"presid"
Topic: 2 
Words: 0.030*"charg" + 0.028*"court" + 0.025*"elect" + 0.024*"polic" + 0.023*"death" + 0.022*"murder" + 0.018*"face" + 0.016*"alleg" + 0.016*"interview" + 0.015*"woman"
Topic: 3 
Words: 0.018*"coast" + 0.014*"donald" + 0.013*"rise" + 0.012*"price" + 0.012*"gold" + 0.011*"victoria" + 0.010*"health" + 0.010*"budget" + 0.010*"centr" + 0.010*"farm"
Topic: 4 
Words: 0.067*"australia" + 0.035*"australian" + 0.028*"south" + 0.027*"kill" + 0.020*"attack" + 0.018*"hour" + 0.017*"west" + 0.011*"record" + 0.011*"drum" + 0.009*"weather"
Topic: 5 
Words: 0.024*"nation" + 0.021*"call" + 0.019*"plan" + 0.015*"water" + 0.014*"state" + 0.01

In [23]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [24]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.009*"ash" + 0.008*"david" + 0.007*"thursday" + 0.007*"festiv" + 0.006*"toni" + 0.006*"andrew" + 0.006*"scott" + 0.005*"interview" + 0.005*"wallabi" + 0.005*"william"
Topic: 1 Word: 0.012*"drum" + 0.011*"elect" + 0.011*"christma" + 0.008*"refuge" + 0.007*"asylum" + 0.007*"stori" + 0.007*"tuesday" + 0.006*"labor" + 0.006*"seeker" + 0.006*"island"
Topic: 2 Word: 0.018*"charg" + 0.018*"polic" + 0.015*"murder" + 0.011*"court" + 0.011*"woman" + 0.010*"death" + 0.010*"alleg" + 0.010*"jail" + 0.009*"assault" + 0.009*"shoot"
Topic: 3 Word: 0.012*"interview" + 0.009*"leagu" + 0.008*"australia" + 0.008*"hobart" + 0.008*"world" + 0.008*"final" + 0.007*"miss" + 0.007*"search" + 0.006*"beat" + 0.006*"rugbi"
Topic: 4 Word: 0.024*"trump" + 0.014*"queensland" + 0.012*"weather" + 0.011*"donald" + 0.010*"south" + 0.009*"north" + 0.008*"korea" + 0.007*"friday" + 0.006*"cyclon" + 0.006*"smith"
Topic: 5 Word: 0.012*"child" + 0.010*"royal" + 0.009*"abus" + 0.009*"sport" + 0.007*"commiss" + 0

In [25]:
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [26]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.8199616074562073	 
Topic: 0.040*"year" + 0.027*"adelaid" + 0.015*"hous" + 0.015*"life" + 0.014*"fall" + 0.013*"near" + 0.013*"sentenc" + 0.011*"young" + 0.010*"refuge" + 0.010*"island"

Score: 0.02000613324344158	 
Topic: 0.024*"nation" + 0.021*"call" + 0.019*"plan" + 0.015*"water" + 0.014*"state" + 0.012*"park" + 0.011*"releas" + 0.011*"prison" + 0.010*"children" + 0.010*"review"

Score: 0.02000543288886547	 
Topic: 0.035*"say" + 0.031*"govern" + 0.020*"chang" + 0.016*"rural" + 0.013*"minist" + 0.012*"farmer" + 0.011*"support" + 0.011*"need" + 0.011*"labor" + 0.010*"feder"

Score: 0.02000490389764309	 
Topic: 0.018*"coast" + 0.014*"donald" + 0.013*"rise" + 0.012*"price" + 0.012*"gold" + 0.011*"victoria" + 0.010*"health" + 0.010*"budget" + 0.010*"centr" + 0.010*"farm"

Score: 0.02000461146235466	 
Topic: 0.033*"sydney" + 0.029*"queensland" + 0.021*"crash" + 0.021*"canberra" + 0.019*"die" + 0.018*"brisban" + 0.018*"final" + 0.018*"live" + 0.014*"turnbul" + 0.013*"concern"

Sco

In [27]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5796586275100708	 
Topic: 0.026*"countri" + 0.024*"hour" + 0.012*"podcast" + 0.011*"turnbul" + 0.008*"violenc" + 0.007*"octob" + 0.007*"domest" + 0.007*"malcolm" + 0.006*"mount" + 0.006*"grand"

Score: 0.2602965235710144	 
Topic: 0.016*"rural" + 0.014*"market" + 0.012*"news" + 0.010*"price" + 0.008*"share" + 0.007*"farmer" + 0.007*"farm" + 0.007*"rise" + 0.006*"dollar" + 0.006*"busi"

Score: 0.020007964223623276	 
Topic: 0.009*"govern" + 0.008*"fund" + 0.007*"health" + 0.007*"budget" + 0.006*"servic" + 0.006*"plan" + 0.005*"council" + 0.005*"indigen" + 0.005*"say" + 0.005*"septemb"

Score: 0.02000744268298149	 
Topic: 0.024*"trump" + 0.014*"queensland" + 0.012*"weather" + 0.011*"donald" + 0.010*"south" + 0.009*"north" + 0.008*"korea" + 0.007*"friday" + 0.006*"cyclon" + 0.006*"smith"

Score: 0.020005596801638603	 
Topic: 0.018*"charg" + 0.018*"polic" + 0.015*"murder" + 0.011*"court" + 0.011*"woman" + 0.010*"death" + 0.010*"alleg" + 0.010*"jail" + 0.009*"assault" + 0.009*"shoot

In [28]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.3502257168292999	 Topic: 0.021*"open" + 0.020*"market" + 0.018*"women" + 0.017*"miss" + 0.016*"australian"
Score: 0.18345728516578674	 Topic: 0.034*"polic" + 0.024*"home" + 0.023*"warn" + 0.020*"china" + 0.017*"north"
Score: 0.18333959579467773	 Topic: 0.033*"sydney" + 0.029*"queensland" + 0.021*"crash" + 0.021*"canberra" + 0.019*"die"
Score: 0.18289288878440857	 Topic: 0.024*"nation" + 0.021*"call" + 0.019*"plan" + 0.015*"water" + 0.014*"state"
Score: 0.016682447865605354	 Topic: 0.035*"say" + 0.031*"govern" + 0.020*"chang" + 0.016*"rural" + 0.013*"minist"
Score: 0.016680413857102394	 Topic: 0.034*"trump" + 0.026*"melbourn" + 0.019*"world" + 0.018*"countri" + 0.011*"abus"
Score: 0.016680413857102394	 Topic: 0.030*"charg" + 0.028*"court" + 0.025*"elect" + 0.024*"polic" + 0.023*"death"
Score: 0.016680413857102394	 Topic: 0.018*"coast" + 0.014*"donald" + 0.013*"rise" + 0.012*"price" + 0.012*"gold"
Score: 0.016680413857102394	 Topic: 0.067*"australia" + 0.035*"australian" + 0.028